part 1.1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve2d
from skimage import io, color

# 加载灰度图像
image = io.imread("F:/UCB/CS180/project2/origin_photo/cameraman.png")

# 检查图像是否有4个通道（RGBA）
if image.shape[-1] == 4:
    image = image[:, :, :3]  # 去掉alpha通道

# 将图像转换为灰度图像
gray_image = color.rgb2gray(image)

# 定义有限差分算子
D_x = np.array([[1, -1]])  # 检测x方向的变化
D_y = np.array([[1], [-1]])  # 检测y方向的变化

# 将图像与D_x和D_y进行卷积
grad_x = convolve2d(gray_image, D_x, mode='same', boundary='symm')
grad_y = convolve2d(gray_image, D_y, mode='same', boundary='symm')

# 计算梯度幅度
gradient_magnitude = np.sqrt(grad_x**2 + grad_y**2)

# 显示梯度幅度图像
plt.imshow(gradient_magnitude, cmap='gray')
plt.title('Gradient magnitude')
plt.show()

# 对梯度幅度进行二值化处理，设定阈值为0.1（可调整）
threshold = 0.2        # try different values
edge_image = gradient_magnitude > threshold

# 显示二值化后的边缘图像
plt.imshow(edge_image, cmap='gray')
plt.title('edge image')
plt.show()


part 1.2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve2d
from skimage import io, color
import cv2

# 加载图像
image = io.imread("F:/UCB/CS180/project2/origin_photo/cameraman.png")
# 如果图像有透明通道，移除alpha通道
if image.shape[-1] == 4:
    image = image[:, :, :3]

# 转换为灰度图像
gray_image = color.rgb2gray(image)

# 创建 1D 高斯核
kernel_size = 5  # 核大小
sigma = 1.0  # 高斯标准差
gaussian_1d = cv2.getGaussianKernel(kernel_size, sigma)

# 生成 2D 高斯核
gaussian_2d = np.outer(gaussian_1d, gaussian_1d.T)

# 对图像进行高斯模糊处理
blurred_image = convolve2d(gray_image, gaussian_2d, mode='same', boundary='symm')

# 显示模糊后的图像
plt.imshow(blurred_image, cmap='gray')
plt.title('Gaussian Blurred Image')
plt.show()

# 定义有限差分算子
D_x = np.array([[1, -1]])  # 检测x方向的变化
D_y = np.array([[1], [-1]])  # 检测y方向的变化

# 将模糊图像与 D_x 和 D_y 卷积
grad_x_blurred = convolve2d(blurred_image, D_x, mode='same', boundary='symm')
grad_y_blurred = convolve2d(blurred_image, D_y, mode='same', boundary='symm')

# 计算梯度幅度
gradient_magnitude_blurred = np.sqrt(grad_x_blurred**2 + grad_y_blurred**2)

# 显示梯度幅度图像
plt.imshow(gradient_magnitude_blurred, cmap='gray')
plt.title('Gradient Magnitude after Gaussian Blurring')
plt.show()

# 生成 DoG 滤波器
DoG_x = convolve2d(gaussian_2d, D_x, mode='full')
DoG_y = convolve2d(gaussian_2d, D_y, mode='full')

# 显示 DoG 滤波器
plt.subplot(1, 2, 1)
plt.imshow(DoG_x, cmap='gray')
plt.title('DoG_x')
plt.subplot(1, 2, 2)
plt.imshow(DoG_y, cmap='gray')
plt.title('DoG_y')
plt.show()

# 使用 DoG 滤波器卷积原始图像
grad_x_dog = convolve2d(gray_image, DoG_x, mode='same', boundary='symm')
grad_y_dog = convolve2d(gray_image, DoG_y, mode='same', boundary='symm')

# 计算梯度幅度
gradient_magnitude_dog = np.sqrt(grad_x_dog**2 + grad_y_dog**2)

# 显示梯度幅度图像（使用 DoG）
plt.imshow(gradient_magnitude_dog, cmap='gray')
plt.title('Gradient Magnitude with DoG Filter')
plt.show()


part 2.1 color images

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import io

# 加载两张图片
image1 = io.imread("F:/UCB/CS180/project2/origin_photo/taj.jpg")
image2 = io.imread("F:/UCB/CS180/project2/origin_photo/6.JPG")

# 如果图像有透明通道，移除alpha通道
if image1.shape[-1] == 4:
    image1 = image1[:, :, :3]
if image2.shape[-1] == 4:
    image2 = image2[:, :, :3]

# 定义高斯核大小和标准差
kernel_size = 5
sigma = 2.0

# 对两张图像应用高斯模糊
blurred_image1 = cv2.GaussianBlur(image1, (kernel_size, kernel_size), sigma)
blurred_image2 = cv2.GaussianBlur(image2, (kernel_size, kernel_size), sigma)

# 提取高频分量（原图 - 模糊图像）
high_freq1 = image1 - blurred_image1
high_freq2 = image2 - blurred_image2

# 将高频分量重新添加回原图，实现锐化
alpha = 2  # 控制高频分量的添加比例
sharpened_image1 = cv2.addWeighted(image1, 1 + alpha, blurred_image1, -alpha, 0)
sharpened_image2 = cv2.addWeighted(image2, 1 + alpha, blurred_image2, -alpha, 0)

# 显示原图、模糊图像、锐化图像
plt.figure(figsize=(12, 8))

# 第一张图像结果
plt.subplot(2, 3, 1)
plt.imshow(image1)
plt.title('Original Image 1')

plt.subplot(2, 3, 2)
plt.imshow(blurred_image1)
plt.title('Blurred Image 1')

plt.subplot(2, 3, 3)
plt.imshow(sharpened_image1)
plt.title('Sharpened Image 1')

# 第二张图像结果
plt.subplot(2, 3, 4)
plt.imshow(image2)
plt.title('Original Image 2')

plt.subplot(2, 3, 5)
plt.imshow(blurred_image2)
plt.title('Blurred Image 2')

plt.subplot(2, 3, 6)
plt.imshow(sharpened_image2)
plt.title('Sharpened Image 2')

# 调整子图的整体布局，使空白均匀
plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05, wspace=0.2, hspace=0.3)

plt.tight_layout()
plt.show()


part 2.1 gray scale

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, color, exposure

# 加载两张图片
image1 = io.imread("F:/UCB/CS180/project2/origin_photo/taj.jpg")
image2 = io.imread("F:/UCB/CS180/project2/origin_photo/6.JPG")

# 如果图像有透明通道，移除alpha通道
if image1.shape[-1] == 4:
    image1 = image1[:, :, :3]
if image2.shape[-1] == 4:
    image2 = image2[:, :, :3]

# 将图像转换为灰度图像
gray_image1 = color.rgb2gray(image1)
gray_image2 = color.rgb2gray(image2)

# 定义高斯核大小和标准差
kernel_size = 5
sigma = 2.0

# 对两张图像应用高斯模糊
blurred_image1 = cv2.GaussianBlur(gray_image1, (kernel_size, kernel_size), sigma)
blurred_image2 = cv2.GaussianBlur(gray_image2, (kernel_size, kernel_size), sigma)

# 提取高频分量（原图 - 模糊图像）
high_freq1 = gray_image1 - blurred_image1
high_freq2 = gray_image2 - blurred_image2

# 将高频分量重新添加回原图，实现锐化
alpha = 2  # 控制高频分量的添加比例
sharpened_image1 = gray_image1 + alpha * high_freq1
sharpened_image2 = gray_image2 + alpha * high_freq2

# 调整锐化后图像的亮度，使之和原图相同
sharpened_image1_bright = exposure.match_histograms(sharpened_image1, gray_image1)
sharpened_image2_bright = exposure.match_histograms(sharpened_image2, gray_image2)

# 显示原图、模糊图像、锐化图像、以及亮度调整后的锐化图像
plt.figure(figsize=(12, 8))

# 第一张图像结果
plt.subplot(2, 4, 1)
plt.imshow(gray_image1, cmap='gray')
plt.title('Original Image 1')

plt.subplot(2, 4, 2)
plt.imshow(blurred_image1, cmap='gray')
plt.title('Blurred Image 1')

plt.subplot(2, 4, 3)
plt.imshow(sharpened_image1, cmap='gray')
plt.title('Sharpened Image 1')

plt.subplot(2, 4, 4)
plt.imshow(sharpened_image1_bright, cmap='gray')
plt.title('Brightness Adjusted Image 1')

# 第二张图像结果
plt.subplot(2, 4, 5)
plt.imshow(gray_image2, cmap='gray')
plt.title('Original Image 2')

plt.subplot(2, 4, 6)
plt.imshow(blurred_image2, cmap='gray')
plt.title('Blurred Image 2')

plt.subplot(2, 4, 7)
plt.imshow(sharpened_image2, cmap='gray')
plt.title('Sharpened Image 2')

plt.subplot(2, 4, 8)
plt.imshow(sharpened_image2_bright, cmap='gray')
plt.title('Brightness Adjusted Image 2')

plt.tight_layout()
plt.show()


2.1 sharpen blurry image

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import io

# 加载模糊图像
image = io.imread("F:/UCB/CS180/project2/origin_photo/2.png")

# 应用高斯模糊（低通滤波器）
kernel_size = 5 # 高斯核大小
sigma = 1.0  # 高斯模糊的标准差
blurred_image = cv2.GaussianBlur(image, (kernel_size, kernel_size), sigma)

# 提取高频分量（原图 - 模糊图像）
high_freq = image - blurred_image

# 将高频分量重新添加回原图，实现锐化
alpha = 5  # 控制高频分量的添加比例
sharpened_image = cv2.addWeighted(image, 1 + alpha, blurred_image, -alpha, 0)

# 显示原图和锐化图像对比
plt.figure(figsize=(12, 6))

# 原始模糊图像
plt.subplot(1, 2, 1)
plt.imshow(image)
plt.title('Original Blurry Image')

# 锐化后的图像
plt.subplot(1, 2, 2)
plt.imshow(sharpened_image)
plt.title('Sharpened Image')

# 优化图像布局
plt.tight_layout()
plt.show()

part 2.2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
from utils import align_images
from skimage import color 

# 加载图像
im1 = plt.imread('F:/UCB/CS180/project2/origin_photo/DerekPicture.jpg') / 255.  # 低频图像
im2 = plt.imread('F:/UCB/CS180/project2/origin_photo/nutmeg.jpg') / 255.        # 高频图像

# 对齐两张图像
im2_aligned, im1_aligned = align_images(im2, im1)

# 灰度图像
im1_aligned = color.rgb2gray(im1_aligned)
im2_aligned = color.rgb2gray(im2_aligned)

# 定义生成混合图像的函数
def hybrid_image(im1, im2, sigma_low, sigma_high):
    # 低频部分：对 im1 应用高斯模糊
    low_frequencies = cv2.GaussianBlur(im1, (0, 0), sigma_low)
    
    # 高频部分：从 im2 中减去模糊部分，保留高频
    high_frequencies = im2 - cv2.GaussianBlur(im2, (0, 0), sigma_high)
    
    # 将低频和高频信息相加生成混合图像
    hybrid = np.clip(low_frequencies + high_frequencies, 0, 1)
    return hybrid

# 定义高斯模糊的标准差
sigma1 = 3  # 低频滤波器的 sigma 值
sigma2 = 21  # 高频滤波器的 sigma 值

# 生成混合图像
hybrid = hybrid_image(im1_aligned, im2_aligned, sigma1, sigma2)

# 显示混合图像
plt.imshow(hybrid, cmap='gray')
plt.title('Hybrid Image')
plt.show()

# 计算并显示傅里叶变换频谱
def display_fft(image, title):
    # 直接对灰度图像计算傅里叶变换，不需要转换颜色空间
    f_transform = np.fft.fftshift(np.fft.fft2(image))
    magnitude_spectrum = 20 * np.log(np.abs(f_transform) + 1)  # 避免 log(0)

    plt.imshow(magnitude_spectrum, cmap='gray')
    plt.title(title)
    plt.show()

# 展示输入图像、混合图像的频率分析
display_fft(im1_aligned, "FFT of Image 1 (Low Frequencies)")
display_fft(im2_aligned, "FFT of Image 2 (High Frequencies)")
display_fft(hybrid, "FFT of Hybrid Image")


part 2.3

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Function to create Gaussian Stack per channel
def gaussian_stack(image, num_levels, sigma):
    stack = [image]  # First level is the original image
    for i in range(1, num_levels):
        blurred_image = cv2.GaussianBlur(stack[-1], (0, 0), sigma)
        stack.append(blurred_image)
    return stack

# Function to create Laplacian Stack per channel
def laplacian_stack(gaussian_stack):
    laplacian_stack = []
    for i in range(len(gaussian_stack) - 1):
        # Subtract the next Gaussian level from the current one to get Laplacian
        laplacian = cv2.subtract(gaussian_stack[i], gaussian_stack[i + 1])
        laplacian_stack.append(laplacian)
    laplacian_stack.append(gaussian_stack[-1])  # Last level is the lowest-resolution Gaussian
    return laplacian_stack

# Function to process color images by applying stack operations to each channel
def process_color_image(image, num_levels, sigma):
    # Split the color image into its R, G, B channels
    channels = cv2.split(image)
    
    # Process each channel separately
    gaussian_stacks = []
    laplacian_stacks = []
    for channel in channels:
        gauss_stack = gaussian_stack(channel, num_levels, sigma)
        lap_stack = laplacian_stack(gauss_stack)
        gaussian_stacks.append(gauss_stack)
        laplacian_stacks.append(lap_stack)
    
    # Merge channels back after processing
    gaussian_merged = [cv2.merge([g[i] for g in gaussian_stacks]) for i in range(num_levels)]
    laplacian_merged = [cv2.merge([l[i] for l in laplacian_stacks]) for i in range(num_levels)]
    
    return gaussian_merged, laplacian_merged

# Function to normalize the image intensities to the range [0, 255] and convert to uint8
def normalize_image(image):
    norm_image = np.zeros_like(image)
    cv2.normalize(image, norm_image, 0, 255, cv2.NORM_MINMAX)
    return norm_image.astype(np.uint8)

# Display both Gaussian and Laplacian stacks in a single image
def display_stacks(gaussian_stack, laplacian_stack):
    num_levels = len(gaussian_stack)
    plt.figure(figsize=(20, 10))  # Adjust figure size to fit both stacks
    for i in range(num_levels):
        # Normalize Gaussian stack and display
        plt.subplot(2, num_levels, i + 1)
        gaussian_image = normalize_image(gaussian_stack[i])
        plt.imshow(gaussian_image)
        plt.title(f'Gaussian Level {i + 1}')
        plt.axis('off')
        
        # Normalize Laplacian stack and display
        plt.subplot(2, num_levels, i + num_levels + 1)
        laplacian_image = normalize_image(laplacian_stack[i])
        plt.imshow(laplacian_image)
        plt.title(f'Laplacian Level {i + 1}')
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

# Load the image (in color, convert to float32 for processing)
image = cv2.imread('F:/UCB/CS180/project2/origin_photo/5.jpg').astype(np.float32) / 255.0
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for correct display

# Parameters
num_levels = 5  # Number of levels in the stack
sigma = 2  # Sigma value for Gaussian smoothing

# Process color image (per channel)
gaussian_stack, laplacian_stack = process_color_image(image, num_levels, sigma)

# Display both stacks in a single figure
display_stacks(gaussian_stack, laplacian_stack)


part 2.4 regular mask

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

# 创建高斯堆栈的函数
def gaussian_stack(image, num_levels, sigma):
    stack = [image]  # 第一层为原始图像
    for i in range(1, num_levels):
        blurred_image = cv2.GaussianBlur(stack[-1], (0, 0), sigma)
        downsampled_image = cv2.pyrDown(blurred_image)
        stack.append(downsampled_image)
    return stack

# 创建拉普拉斯堆栈的函数
def laplacian_stack(gaussian_stack):
    laplacian_stack = []
    for i in range(len(gaussian_stack) - 1):
        gaussian_expanded = cv2.pyrUp(gaussian_stack[i + 1], dstsize=(gaussian_stack[i].shape[1], gaussian_stack[i].shape[0]))
        laplacian = cv2.subtract(gaussian_stack[i], gaussian_expanded)
        laplacian_stack.append(laplacian)
    laplacian_stack.append(gaussian_stack[-1])  # 最后一层是最低分辨率的高斯图像
    return laplacian_stack

# 处理彩色图像，将堆栈操作应用到每个通道
def process_color_image(image, num_levels, sigma):
    channels = cv2.split(image)
    
    gaussian_stacks = []
    laplacian_stacks = []
    for channel in channels:
        gauss_stack = gaussian_stack(channel, num_levels, sigma)
        lap_stack = laplacian_stack(gauss_stack)
        gaussian_stacks.append(gauss_stack)
        laplacian_stacks.append(lap_stack)
    
    # 处理后将各通道合并回彩色图像
    gaussian_merged = [cv2.merge([g[i] for g in gaussian_stacks]) for i in range(num_levels)]
    laplacian_merged = [cv2.merge([l[i] for l in laplacian_stacks]) for i in range(num_levels)]
    
    return gaussian_merged, laplacian_merged

# 归一化图像到 0-255 范围，并转换为 uint8
def normalize_image(image):
    norm_image = np.zeros_like(image)
    cv2.normalize(image, norm_image, 0, 255, cv2.NORM_MINMAX)
    return norm_image.astype(np.uint8)

# 多分辨率图像混合，使用掩码
def multiresolution_blending(image1, image2, mask, num_levels, sigma):
    # 生成图像的高斯和拉普拉斯堆栈
    gauss1, laplacian1 = process_color_image(image1, num_levels, sigma)
    gauss2, laplacian2 = process_color_image(image2, num_levels, sigma)
    
    # 生成掩码的高斯堆栈并扩展维度以匹配彩色图像
    mask_stack = gaussian_stack(mask, num_levels, sigma)
    mask_stack = [cv2.merge([mask_level, mask_level, mask_level]) for mask_level in mask_stack]  # 将掩码扩展为3通道
    
    # 初始化混合图像堆栈
    blended_stack = []
    
    # 在每一层混合图像
    for i in range(num_levels):
        blended = laplacian1[i] * mask_stack[i] + laplacian2[i] * (1 - mask_stack[i])
        blended_stack.append(blended)
    
    # 重建最终的混合图像
    blended_image = blended_stack[-1]
    for i in range(num_levels - 2, -1, -1):
        # 上采样并调整尺寸以匹配当前层的大小
        upsampled = cv2.pyrUp(blended_image)
        if upsampled.shape != blended_stack[i].shape:
            upsampled = cv2.resize(upsampled, (blended_stack[i].shape[1], blended_stack[i].shape[0]))
        blended_image = cv2.add(upsampled, blended_stack[i])
    
    return blended_image

# 读取图像
image1 = cv2.imread('F:/UCB/CS180/project2/origin_photo/apple.jpeg').astype(np.float32) / 255.0
image2 = cv2.imread('F:/UCB/CS180/project2/origin_photo/orange.jpeg').astype(np.float32) / 255.0
image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)

# 创建二值掩码，左边为1，右边为0
mask = np.zeros_like(image1[:, :, 0], dtype=np.float32)
mask[:, :mask.shape[1] // 2] = 1

# 高斯堆栈和拉普拉斯堆栈的参数
num_levels = 6  # 增加堆栈层级，提升平滑度
sigma = 1.5     # 调整模糊强度

# 多分辨率混合图像
blended_image = multiresolution_blending(image1, image2, mask, num_levels, sigma)

# 显示结果
plt.imshow(np.clip(blended_image, 0, 1))
plt.title('Blended Image')
plt.axis('off')
plt.show()


part 2.4 layout the layers

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

# 创建高斯堆栈的函数
def gaussian_stack(image, num_levels, sigma):
    stack = [image]  # 第一层为原始图像
    for i in range(1, num_levels):
        blurred_image = cv2.GaussianBlur(stack[-1], (0, 0), sigma)
        downsampled_image = cv2.pyrDown(blurred_image)
        stack.append(downsampled_image)
    return stack

# 创建拉普拉斯堆栈的函数
def laplacian_stack(gaussian_stack):
    laplacian_stack = []
    for i in range(len(gaussian_stack) - 1):
        gaussian_expanded = cv2.pyrUp(gaussian_stack[i + 1], dstsize=(gaussian_stack[i].shape[1], gaussian_stack[i].shape[0]))
        laplacian = cv2.subtract(gaussian_stack[i], gaussian_expanded)
        laplacian_stack.append(laplacian)
    laplacian_stack.append(gaussian_stack[-1])  # 最后一层是最低分辨率的高斯图像
    return laplacian_stack

# 处理彩色图像，将堆栈操作应用到每个通道
def process_color_image(image, num_levels, sigma):
    channels = cv2.split(image)
    
    gaussian_stacks = []
    laplacian_stacks = []
    for channel in channels:
        gauss_stack = gaussian_stack(channel, num_levels, sigma)
        lap_stack = laplacian_stack(gauss_stack)
        gaussian_stacks.append(gauss_stack)
        laplacian_stacks.append(lap_stack)
    
    # 处理后将各通道合并回彩色图像
    gaussian_merged = [cv2.merge([g[i] for g in gaussian_stacks]) for i in range(num_levels)]
    laplacian_merged = [cv2.merge([l[i] for l in laplacian_stacks]) for i in range(num_levels)]
    
    return gaussian_merged, laplacian_merged

# 一次性展示所有层的高斯和拉普拉斯堆栈图像
def display_all_layers(gaussian_merged, laplacian_merged, num_levels):
    plt.figure(figsize=(20, 10))  # 调整画布大小
    for i in range(num_levels):
        # 高斯堆栈
        plt.subplot(2, num_levels, i + 1)
        plt.imshow(np.clip(gaussian_merged[i], 0, 1))
        plt.title(f'Gaussian Level {i + 1}')
        plt.axis('off')

        # 拉普拉斯堆栈
        plt.subplot(2, num_levels, i + num_levels + 1)
        plt.imshow(np.clip(laplacian_merged[i], 0, 1))
        plt.title(f'Laplacian Level {i + 1}')
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

# 多分辨率图像混合，使用掩码
def multiresolution_blending(image1, image2, mask, num_levels, sigma):
    # 生成图像的高斯和拉普拉斯堆栈
    gauss1, laplacian1 = process_color_image(image1, num_levels, sigma)
    gauss2, laplacian2 = process_color_image(image2, num_levels, sigma)
    
    # 生成掩码的高斯堆栈并扩展维度以匹配彩色图像
    mask_stack = gaussian_stack(mask, num_levels, sigma)
    mask_stack = [cv2.merge([mask_level, mask_level, mask_level]) for mask_level in mask_stack]  # 将掩码扩展为3通道
    
    # 初始化混合图像堆栈
    blended_stack = []
    
    # 在每一层混合图像
    for i in range(num_levels):
        blended = laplacian1[i] * mask_stack[i] + laplacian2[i] * (1 - mask_stack[i])
        blended_stack.append(blended)
    
    # 重建最终的混合图像
    blended_image = blended_stack[-1]
    for i in range(num_levels - 2, -1, -1):
        # 上采样并调整尺寸以匹配当前层的大小
        upsampled = cv2.pyrUp(blended_image)
        if upsampled.shape != blended_stack[i].shape:
            upsampled = cv2.resize(upsampled, (blended_stack[i].shape[1], blended_stack[i].shape[0]))
        blended_image = cv2.add(upsampled, blended_stack[i])
    
    return blended_image

# 读取图像
image1 = cv2.imread('F:/UCB/CS180/project2/origin_photo/apple.jpeg').astype(np.float32) / 255.0
image2 = cv2.imread('F:/UCB/CS180/project2/origin_photo/orange.jpeg').astype(np.float32) / 255.0
image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)

# 创建二值掩码，左边为1，右边为0
mask = np.zeros_like(image1[:, :, 0], dtype=np.float32)
mask[:, :mask.shape[1] // 2] = 1

# 高斯堆栈和拉普拉斯堆栈的参数
num_levels = 6  # 增加堆栈层级，提升平滑度
sigma = 1.5     # 调整模糊强度

# 生成高斯和拉普拉斯堆栈
gaussian_merged1, laplacian_merged1 = process_color_image(image1, num_levels, sigma)

# 显示所有高斯和拉普拉斯层
display_all_layers(gaussian_merged1, laplacian_merged1, num_levels)

# 多分辨率混合图像
blended_image = multiresolution_blending(image1, image2, mask, num_levels, sigma)

# 显示最终混合结果
plt.imshow(np.clip(blended_image, 0, 1))
plt.title('Blended Image')
plt.axis('off')
plt.show()


part 2.4 irrgular mask

In [ ]:
# # import numpy as np
# # import cv2
# # import matplotlib.pyplot as plt

# # # 创建高斯堆栈的函数
# # def gaussian_stack(image, num_levels, sigma):
# #     stack = [image]  # 第一层为原始图像
# #     for i in range(1, num_levels):
# #         blurred_image = cv2.GaussianBlur(stack[-1], (0, 0), sigma)
# #         downsampled_image = cv2.pyrDown(blurred_image)
# #         stack.append(downsampled_image)
# #     return stack

# # # 创建拉普拉斯堆栈的函数
# # def laplacian_stack(gaussian_stack):
# #     laplacian_stack = []
# #     for i in range(len(gaussian_stack) - 1):
# #         gaussian_expanded = cv2.pyrUp(gaussian_stack[i + 1], dstsize=(gaussian_stack[i].shape[1], gaussian_stack[i].shape[0]))
# #         laplacian = cv2.subtract(gaussian_stack[i], gaussian_expanded)
# #         laplacian_stack.append(laplacian)
# #     laplacian_stack.append(gaussian_stack[-1])  # 最后一层是最低分辨率的高斯图像
# #     return laplacian_stack

# # # 处理彩色图像，将堆栈操作应用到每个通道
# # def process_color_image(image, num_levels, sigma):
# #     channels = cv2.split(image)
    
# #     gaussian_stacks = []
# #     laplacian_stacks = []
# #     for channel in channels:
# #         gauss_stack = gaussian_stack(channel, num_levels, sigma)
# #         lap_stack = laplacian_stack(gauss_stack)
# #         gaussian_stacks.append(gauss_stack)
# #         laplacian_stacks.append(lap_stack)
    
# #     # 处理后将各通道合并回彩色图像
# #     gaussian_merged = [cv2.merge([g[i] for g in gaussian_stacks]) for i in range(num_levels)]
# #     laplacian_merged = [cv2.merge([l[i] for l in laplacian_stacks]) for i in range(num_levels)]
    
# #     return gaussian_merged, laplacian_merged

# # # 多分辨率图像混合，使用掩码
# # def multiresolution_blending(image1, image2, mask, num_levels, sigma):
# #     # 生成图像的高斯和拉普拉斯堆栈
# #     gauss1, laplacian1 = process_color_image(image1, num_levels, sigma)
# #     gauss2, laplacian2 = process_color_image(image2, num_levels, sigma)
    
# #     # 生成掩码的高斯堆栈并扩展维度以匹配彩色图像
# #     mask_stack = gaussian_stack(mask, num_levels, sigma)
# #     mask_stack = [cv2.merge([mask_level, mask_level, mask_level]) for mask_level in mask_stack]  # 将掩码扩展为3通道
    
# #     # 初始化混合图像堆栈
# #     blended_stack = []
    
# #     # 在每一层混合图像
# #     for i in range(num_levels):
# #         blended = laplacian1[i] * mask_stack[i] + laplacian2[i] * (1 - mask_stack[i])
# #         blended_stack.append(blended)
    
# #     # 重建最终的混合图像
# #     blended_image = blended_stack[-1]
# #     for i in range(num_levels - 2, -1, -1):
# #         # 上采样并调整尺寸以匹配当前层的大小
# #         upsampled = cv2.pyrUp(blended_image)
# #         if upsampled.shape != blended_stack[i].shape:
# #             upsampled = cv2.resize(upsampled, (blended_stack[i].shape[1], blended_stack[i].shape[0]))
# #         blended_image = cv2.add(upsampled, blended_stack[i])
    
# #     return blended_image

# # # 读取图像
# # image1 = cv2.imread('F:/UCB/CS180/project2/origin_photo/12.png').astype(np.float32) / 255.0
# # image2 = cv2.imread('F:/UCB/CS180/project2/origin_photo/13.png').astype(np.float32) / 255.0
# # mask = cv2.imread('F:/UCB/CS180/project2/origin_photo/14.png').astype(np.float32) / 255.0

# # image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
# # image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)

# # # 讲三张图片裁剪成最小的图片
# # min_width = min(image1.shape[1], image2.shape[1], mask.shape[1])
# # min_height = min(image1.shape[0], image2.shape[0], mask.shape[0])

# # image1 = cv2.resize(image1, (min_width, min_height))
# # image2 = cv2.resize(image2, (min_width, min_height))
# # mask = cv2.resize(mask, (min_width, min_height))

# # # 读取不规则掩码
# # # mask = cv2.imread('F:/UCB/CS180/project2/origin_photo/mask.png', cv2.IMREAD_GRAYSCALE).astype(np.float32) / 255.0

# # # 把mask变成二值图像
# # mask[mask > 0.5] = 1
# # mask[mask <= 0.5] = 0

# # # 高斯堆栈和拉普拉斯堆栈的参数
# # num_levels = 6  # 增加堆栈层级，提升平滑度
# # sigma = 1.5     # 调整模糊强度

# # # 多分辨率混合图像
# # blended_image = multiresolution_blending(image1, image2, mask, num_levels, sigma)

# # # 显示结果
# # plt.imshow(np.clip(blended_image, 0, 1))
# # plt.title('Blended Image with Irregular Mask')
# # plt.axis('off')
# # plt.show()

# import numpy as np
# import cv2
# import matplotlib.pyplot as plt


# # 创建高斯堆栈的函数
# def gaussian_stack(image, num_levels, sigma):
#     stack = [image]  # 第一层为原始图像
#     for i in range(1, num_levels):
#         blurred_image = cv2.GaussianBlur(stack[-1], (0, 0), sigma)
#         downsampled_image = cv2.pyrDown(blurred_image)
#         stack.append(downsampled_image)
#     return stack

# # 创建拉普拉斯堆栈的函数
# def laplacian_stack(gaussian_stack):
#     laplacian_stack = []
#     for i in range(len(gaussian_stack) - 1):
#         gaussian_expanded = cv2.pyrUp(gaussian_stack[i + 1], dstsize=(gaussian_stack[i].shape[1], gaussian_stack[i].shape[0]))
#         laplacian = cv2.subtract(gaussian_stack[i], gaussian_expanded)
#         laplacian_stack.append(laplacian)
#     laplacian_stack.append(gaussian_stack[-1])  # 最后一层是最低分辨率的高斯图像
#     return laplacian_stack

# # 处理彩色图像，将堆栈操作应用到每个通道
# def process_color_image(image, num_levels, sigma):
#     channels = cv2.split(image)
    
#     gaussian_stacks = []
#     laplacian_stacks = []
#     for channel in channels:
#         gauss_stack = gaussian_stack(channel, num_levels, sigma)
#         lap_stack = laplacian_stack(gauss_stack)
#         gaussian_stacks.append(gauss_stack)
#         laplacian_stacks.append(lap_stack)
    
#     # 处理后将各通道合并回彩色图像
#     gaussian_merged = [cv2.merge([g[i] for g in gaussian_stacks]) for i in range(num_levels)]
#     laplacian_merged = [cv2.merge([l[i] for l in laplacian_stacks]) for i in range(num_levels)]
    
#     return gaussian_merged, laplacian_merged

# # 多分辨率图像混合，使用掩码
# def multiresolution_blending(image1, image2, mask, num_levels, sigma):
#     # 生成图像的高斯和拉普拉斯堆栈
#     gauss1, laplacian1 = process_color_image(image1, num_levels, sigma)
#     gauss2, laplacian2 = process_color_image(image2, num_levels, sigma)
    
#     # 确保掩码是单通道的
#     if len(mask.shape) > 2:
#         mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    
#     # 生成掩码的高斯堆栈
#     mask_stack = gaussian_stack(mask, num_levels, sigma)
    
#     # 初始化混合图像堆栈
#     blended_stack = []
    
#     # 在每一层混合图像
#     for i in range(num_levels):
#         # 确保掩码与图像具有相同的通道数
#         current_mask = np.repeat(mask_stack[i][:, :, np.newaxis], 3, axis=2)
#         blended = laplacian1[i] * current_mask + laplacian2[i] * (1 - current_mask)
#         blended_stack.append(blended)
    
#     # 重建最终的混合图像
#     blended_image = blended_stack[-1]
#     for i in range(num_levels - 2, -1, -1):
#         upsampled = cv2.pyrUp(blended_image)
#         if upsampled.shape != blended_stack[i].shape:
#             upsampled = cv2.resize(upsampled, (blended_stack[i].shape[1], blended_stack[i].shape[0]))
#         blended_image = cv2.add(upsampled, blended_stack[i])
    
#     return blended_image


# # 读取图像
# image1 = cv2.imread('F:/UCB/CS180/project2/origin_photo/12.png')
# image2 = cv2.imread('F:/UCB/CS180/project2/origin_photo/13.png')
# mask = cv2.imread('F:/UCB/CS180/project2/origin_photo/14.png', cv2.IMREAD_GRAYSCALE)

# # 确保所有图像和掩码具有相同的大小
# min_width = min(image1.shape[1], image2.shape[1], mask.shape[1])
# min_height = min(image1.shape[0], image2.shape[0], mask.shape[0])

# image1 = cv2.resize(image1, (min_width, min_height))
# image2 = cv2.resize(image2, (min_width, min_height))
# mask = cv2.resize(mask, (min_width, min_height))

# # 转换图像到浮点型并归一化
# image1 = image1.astype(np.float32) / 255.0
# image2 = image2.astype(np.float32) / 255.0
# mask = mask.astype(np.float32) / 255.0

# # 转换图像到RGB颜色空间
# image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
# image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)

# # 调整掩码
# mask = cv2.threshold(mask, 0.5, 1, cv2.THRESH_BINARY)[1]

# # 高斯堆栈和拉普拉斯堆栈的参数
# num_levels = 6  # 增加堆栈层级，提升平滑度
# sigma = 1.5     # 调整模糊强度

# # 多分辨率混合图像
# blended_image = multiresolution_blending(image1, image2, mask, num_levels, sigma)

# # 显示结果
# plt.figure(figsize=(20, 10))

# plt.subplot(231)
# plt.imshow(image1)
# plt.title('Image 1')
# plt.axis('off')

# plt.subplot(232)
# plt.imshow(image2)
# plt.title('Image 2')
# plt.axis('off')

# plt.subplot(233)
# plt.imshow(mask, cmap='gray')
# plt.title('Mask')
# plt.axis('off')

# plt.subplot(234)
# plt.imshow(np.clip(image1 * mask[:,:,np.newaxis] + image2 * (1-mask[:,:,np.newaxis]), 0, 1))
# plt.title('Simple Blend')
# plt.axis('off')

# plt.subplot(235)
# plt.imshow(np.clip(blended_image, 0, 1))
# plt.title('Multiresolution Blended Image')
# plt.axis('off')

# plt.tight_layout()
# plt.show()
# # # 读取图像
# # image1 = cv2.imread('F:/UCB/CS180/project2/origin_photo/12.png').astype(np.float32) / 255.0
# # image2 = cv2.imread('F:/UCB/CS180/project2/origin_photo/13.png').astype(np.float32) / 255.0
# # mask = cv2.imread('F:/UCB/CS180/project2/origin_photo/14.png', cv2.IMREAD_GRAYSCALE).astype(np.float32) / 255.0

# # image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
# # image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)

# # # 确保所有图像和掩码具有相同的大小
# # min_width = min(image1.shape[1], image2.shape[1], mask.shape[1])
# # min_height = min(image1.shape[0], image2.shape[0], mask.shape[0])

# # image1 = cv2.resize(image1, (min_width, min_height))
# # image2 = cv2.resize(image2, (min_width, min_height))
# # mask = cv2.resize(mask, (min_width, min_height))

# # # 把mask变成二值图像
# # mask[mask > 0.1] = 0
# # mask[mask <= 0.1] = 1

# # # 高斯堆栈和拉普拉斯堆栈的参数
# # num_levels = 6  # 增加堆栈层级，提升平滑度
# # sigma = 1.5     # 调整模糊强度

# # # 多分辨率混合图像
# # blended_image = multiresolution_blending(image1, image2, mask, num_levels, sigma)

# # # 显示结果
# # plt.imshow(np.clip(blended_image, 0, 1))
# # plt.title('Blended Image with Irregular Mask')
# # plt.axis('off')
# # plt.show()

import numpy as np
import cv2
import matplotlib.pyplot as plt

# 创建高斯堆栈的函数
def gaussian_stack(image, num_levels, sigma):
    stack = [image]  # 第一层为原始图像
    for i in range(1, num_levels):
        blurred_image = cv2.GaussianBlur(stack[-1], (0, 0), sigma)
        downsampled_image = cv2.pyrDown(blurred_image)
        stack.append(downsampled_image)
    return stack

# 创建拉普拉斯堆栈的函数
def laplacian_stack(gaussian_stack):
    laplacian_stack = []
    for i in range(len(gaussian_stack) - 1):
        gaussian_expanded = cv2.pyrUp(gaussian_stack[i + 1], dstsize=(gaussian_stack[i].shape[1], gaussian_stack[i].shape[0]))
        laplacian = cv2.subtract(gaussian_stack[i], gaussian_expanded)
        laplacian_stack.append(laplacian)
    laplacian_stack.append(gaussian_stack[-1])  # 最后一层是最低分辨率的高斯图像
    return laplacian_stack

# 处理彩色图像，将堆栈操作应用到每个通道
def process_color_image(image, num_levels, sigma):
    channels = cv2.split(image)
    
    gaussian_stacks = []
    laplacian_stacks = []
    for channel in channels:
        gauss_stack = gaussian_stack(channel, num_levels, sigma)
        lap_stack = laplacian_stack(gauss_stack)
        gaussian_stacks.append(gauss_stack)
        laplacian_stacks.append(lap_stack)
    
    # 处理后将各通道合并回彩色图像
    gaussian_merged = [cv2.merge([g[i] for g in gaussian_stacks]) for i in range(num_levels)]
    laplacian_merged = [cv2.merge([l[i] for l in laplacian_stacks]) for i in range(num_levels)]
    
    return gaussian_merged, laplacian_merged

# 多分辨率图像混合，使用掩码
def multiresolution_blending(image1, image2, mask, num_levels, sigma):
    # 生成图像的高斯和拉普拉斯堆栈
    gauss1, laplacian1 = process_color_image(image1, num_levels, sigma)
    gauss2, laplacian2 = process_color_image(image2, num_levels, sigma)
    
    # 确保掩码是单通道的
    if len(mask.shape) > 2:
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    
    # 生成掩码的高斯堆栈
    mask_stack = gaussian_stack(mask, num_levels, sigma)
    
    # 初始化混合图像堆栈
    blended_stack = []
    
    # 在每一层混合图像
    for i in range(num_levels):
        # 确保掩码与图像具有相同的通道数
        current_mask = np.repeat(mask_stack[i][:, :, np.newaxis], 3, axis=2)
        blended = laplacian1[i] * current_mask + laplacian2[i] * (1 - current_mask)
        blended_stack.append(blended)
    
    # 重建最终的混合图像
    blended_image = blended_stack[-1]
    for i in range(num_levels - 2, -1, -1):
        upsampled = cv2.pyrUp(blended_image)
        if upsampled.shape != blended_stack[i].shape:
            upsampled = cv2.resize(upsampled, (blended_stack[i].shape[1], blended_stack[i].shape[0]))
        blended_image = cv2.add(upsampled, blended_stack[i])
    
    return blended_image

# 读取图像
image1 = cv2.imread('F:/UCB/CS180/project2/origin_photo/12.png')
image2 = cv2.imread('F:/UCB/CS180/project2/origin_photo/13.png')
mask = cv2.imread('F:/UCB/CS180/project2/origin_photo/14.png', cv2.IMREAD_GRAYSCALE)

# 确保所有图像和掩码具有相同的大小
min_width = min(image1.shape[1], image2.shape[1], mask.shape[1])
min_height = min(image1.shape[0], image2.shape[0], mask.shape[0])

image1 = cv2.resize(image1, (min_width, min_height))
image2 = cv2.resize(image2, (min_width, min_height))
mask = cv2.resize(mask, (min_width, min_height))

# 转换图像到浮点型并归一化
image1 = image1.astype(np.float32) / 255.0
image2 = image2.astype(np.float32) / 255.0
mask = mask.astype(np.float32) / 255.0

# 转换图像到RGB颜色空间
image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)

# 调整掩码并反转
mask = cv2.threshold(mask, 0.5, 1, cv2.THRESH_BINARY)[1]
mask = 1 - mask  # 反转掩码

# 高斯堆栈和拉普拉斯堆栈的参数
num_levels = 6  # 增加堆栈层级，提升平滑度
sigma = 1.5     # 调整模糊强度

# 多分辨率混合图像
blended_image = multiresolution_blending(image1, image2, mask, num_levels, sigma)

# 显示结果
plt.figure(figsize=(20, 10))

plt.subplot(231)
plt.imshow(image1)
plt.title('Image 1')
plt.axis('off')

plt.subplot(232)
plt.imshow(image2)
plt.title('Image 2')
plt.axis('off')

plt.subplot(233)
plt.imshow(mask, cmap='gray')
plt.title('Inverted Mask')
plt.axis('off')

plt.subplot(234)
plt.imshow(np.clip(image1 * mask[:,:,np.newaxis] + image2 * (1-mask[:,:,np.newaxis]), 0, 1))
plt.title('Simple Blend')
plt.axis('off')

plt.subplot(235)
plt.imshow(np.clip(blended_image, 0, 1))
plt.title('Multiresolution Blended Image')
plt.axis('off')

plt.tight_layout()
plt.show()